In [1]:
import numpy as np
from utilities.Dataset import *
import pickle
import sys
from typing import List
import torch
import os

In [2]:
participant_data_path = './EEG/number'
processed_data_path = './EEG/processed_data'
split_data_path = './EEG/split_data'

In [3]:
ps = os.listdir(participant_data_path)
participants=[]
for p in ps:
    p= p[:3]
    if p not in participants:
        participants.append(p)
        
participants= sorted(participants)
# participants, len(participants)

In [4]:
load_name=f"userfold_data_scaled_p_dictionary-number"
data_dir = "./EEG/split_data/standard_scaled"
try:
    raw_user_fold= pickle.load(open(os.path.join(data_dir, f"{load_name}.pkl"), "rb"))
except:
    print(f"pickle file does not exist. Use EEG-Preprocess.ipynb and EEG-Split.ipynb to save data setting.")
    sys.exit()

In [5]:
from utilities.userfold_framework import *
from Models.AR_EEG_models import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List



In [ ]:
from torcheeg.models import EEGNet
import Models.model_func as Model_Func
from torch import nn
from Models.multi_models import *

DEVICE= torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

learning_rate = 0.00005
batch_size = 64
n_epochs = 300
transpose_channels=True
participants_dictionary=[]
# participants_online_dictionary=[]
participants_grads_dictionary={}
b_acc_list=[]
c0_acc_list=[]
c1_acc_list=[]

# EPOCH=[
    
# ]
for r in range(5):
    for i in range(len(participants)):

        train_dataloader, val_dataloader, classes, input_dim, class_ratio= user_fold_load(i,
                                                                                          raw_user_fold,
                                                                                          participants,
                                                                                          batch_size=batch_size,
                                                                                          transpose_channels=transpose_channels)

        eegnet= EEGNet(
            chunk_size=input_dim[1],
            num_electrodes=input_dim[0],
            num_classes=classes,
            kernel_1= 32,
            kernel_2=32,
            F1=8,
            F2=16,
            dropout=0.5
        ).to(DEVICE)

        classifier= LSTM_EEGNet_Wrapper(DEVICE, eegnet, input_dim).to(DEVICE)
    #     optimizer= torch.optim.RMSprop(classifier.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)


        criterion= torch.nn.CrossEntropyLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
    #     criterion = nn.NLLLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))

        scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

        saved_dir= f"./EEG/saved_models/Userfold/run{r}"
        model= EEGNet_IE_EEG_Wrapper(
            DEVICE, classifier, input_dim[0]
        ).to(DEVICE)

        train_func= eeg_grad_train
        model.training_procedure(iteration=n_epochs,
                                 train_dataloader=train_dataloader,
                                 val_dataloader=val_dataloader,
                                 print_cycle=2,
                                 path=f"./dictionary/intermdiate_dicts",
                                 loss_func=criterion,
                                 optimiser=optimizer, #scheduler=scheduler,
                                 train_func=train_func
                                )
        if model.epoch == n_epochs+1:
            EPOCH= n_epochs
        else:
            EPOCH= model.epoch

        torch.save(model.state_dict(), 
               os.path.join(
                   saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_TS-e{EPOCH}.pt"
               )
        )
        pickle.dump(model.return_IE_grad(),
                    open(
                        os.path.join(
                            saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_TS-grad_all-e{EPOCH}.pkl"
                        ), "wb")
        )
    # OR
    #     model.load_state_dict(
    #     torch.load(
    #         open(
    #             os.path.join(
    #                 saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-e{n_epochs}.pt"
    #             ), "rb"
    #         )
    #               )
    #     )
    #     model.IE_grad= pickle.load(
    #             open(
    #                 os.path.join(
    #                     saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-grad_all-e{n_epochs}.pkl"
    #                 ),"rb"
    #             )
    #         )


        prediction, dictionary= model.prediction_procedure(val_dataloader, dict_flag=True)

        ys= np.concatenate([y.detach().cpu().numpy() for x, y in val_dataloader])

        c0_acc, c1_acc, b_acc= calculate_accuracy(ys, prediction)
        print("c0_acc", c0_acc, ", c1_acc", c1_acc, ", b_acc", b_acc)
        b_acc_list.append(b_acc)
        c0_acc_list.append(c0_acc)
        c1_acc_list.append(c1_acc)
    #     participants_grads_dictionary[participants[i]]= model.return_IE_grad()
        participants_dictionary.append(dictionary)
    
   

tmp=[]
for i, dictionary in enumerate(participants_dictionary):
    print(f"User {participants[i]} f1: {dictionary['weighted avg']['f1-score']} acc: {dictionary['accuracy']}")
    print(f" c0: {c0_acc_list[i]} c1: {c1_acc_list[i]} bacc: {b_acc_list[i]}")
    tmp.append(dictionary['weighted avg']['f1-score'])

print(f"average {np.mean(tmp)}")
print()
print(np.array(b_acc_list).mean())
print(np.array(c1_acc_list).mean())
print(np.array(c0_acc_list).mean())


Iterations:   0%|                                           | 0/300 [00:00<?, ?it/s]

Epoch  0 , loss 0.6409304282244515


Iterations:   0%|                                   | 1/300 [00:03<15:08,  3.04s/it]

Epoch:  0
t_loss:  0.6409304282244515 , v_loss:  0.6924525698026022
t_acc:  0.6031746031746031 , v_acc:  0.6894409937888198
t_recall:  0.5152166453932832 , v_recall:  0.5
t_prec:  0.5163765004775925 , v_prec:  0.3447204968944099
t_f:  0.514995556131207 , v_f:  0.4080882352941176
////////


Iterations:   1%|▏                                  | 2/300 [00:03<08:52,  1.79s/it]

...

Epoch:  112
t_loss:  0.410289398431778 , v_loss:  0.6195503820975622
t_acc:  0.7069344210856605 , v_acc:  0.7219251336898396
t_recall:  0.5276300755232283 , v_recall:  0.5408260632497274
t_prec:  0.7046040819695814 , v_prec:  0.7757826887661142
t_f:  0.4739332891271061 , v_f:  0.49731182795698925
////////


Iterations:  38%|████████████▌                    | 114/300 [02:12<03:32,  1.14s/it]

Epoch  113 , loss 0.41244292974472047
Epoch  114 , loss 0.4096906167268753


Iterations:  38%|████████████▋                    | 115/300 [02:13<03:57,  1.28s/it]

Epoch:  114
t_loss:  0.4096906167268753 , v_loss:  0.6114639937877655
t_acc:  0.7091308440539692 , v_acc:  0.7112299465240641
t_recall:  0.5329629165157131 , v_recall:  0.5331924754634678
t_prec:  0.7043070341603284 , v_prec:  0.6700418994413408
t_f:  0.4851763463513962 , v_f:  0.4910282258064516
////////


In [ ]:
userfold_results_summary(participants_dictionary, participants)
userfold_classwise_results_summary(participants_dictionary, participants)


In [ ]:
participant_grad={"AUC":[],"ROC":[],"STD":[]}
for i in range(len(participants)):
    g= pickle.load(
            open(
                os.path.join(
                    saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_TS-grad_all-e{n_epochs}.pkl"
                ),"rb"
            )
    )
    
    participant_grad["AUC"].append(
        pd.DataFrame(
            Grad_AUC_with_grad(
                g, n_epochs, np.zeros(len(timestep_labels)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )
    
    participant_grad["ROC"].append(
        pd.DataFrame(
            Grad_ROC_with_grad(
                g, n_epochs, np.zeros(len(timestep_labels)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )
    
    participant_grad["STD"].append(
        pd.DataFrame(
            Grad_STD_with_grad(
                g, n_epochs, np.zeros(len(timestep_labels)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )

In [ ]:
start=-0.5
step=1.5/(188-1)
timestep_labels=[]
for i in range(input_dim[1]):
    timestep_labels.append(round(start+step*i,3))
    
# timestep_labels

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler
plt.figure(figsize=(8,6))
methods=["AUC", "ROC", "STD"]
for i in methods:
    scaler= MinMaxScaler()
    scaled_avg_grad= scaler.fit_transform(np.array(participant_grad[i]).mean(0).reshape(-1,1))
#     df.index= channel_names
#     df.columns=timestep_labels
    
#     scaled= scaler.fit_transform(df.sum().to_numpy().reshape(-1,1))
    plt.plot(scaled_avg_grad)
plt.legend(methods)
plt.xlabel("Timesteps")
plt.axvspan(107,121, color="green", alpha=0.2)
plt.axvspan(112,128, color="red", alpha=0.2)
plt.axvspan(142,150, color="red", alpha=0.2)
plt.axvspan(157,159, color="red", alpha=0.2)
plt.xticks([0,31,62,93,124,155,187],[-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0])
plt.ylabel("Normalised Importance")
plt.margins(x=0)
matplotlib.rcParams.update({"font.size":18})
plt.tight_layout()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler
plt.figure(figsize=(8,6))
methods=["AUC", "ROC", "STD"]
for i in methods:
    scaler= RobustScaler()
    scaled_avg_grad= scaler.fit_transform(np.array(participant_grad[i]).mean(0).reshape(-1,1))
#     df.index= channel_names
#     df.columns=timestep_labels
    
#     scaled= scaler.fit_transform(df.sum().to_numpy().reshape(-1,1))
    plt.plot(scaled_avg_grad)
plt.legend(methods)
plt.xlabel("Timesteps")
plt.axvspan(107,121, color="green", alpha=0.2)
plt.axvspan(112,128, color="red", alpha=0.2)
plt.axvspan(142,150, color="red", alpha=0.2)
plt.axvspan(157,159, color="red", alpha=0.2)
plt.xticks([0,31,62,93,124,155,187],[-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0])
plt.ylabel("Normalised Importance")
plt.margins(x=0)
matplotlib.rcParams.update({"font.size":18})
plt.tight_layout()